In [7]:
from lofar_lst import *
from lofar_planner import *
import os 

os.listdir()

['plotting_function.ipynb',
 '.DS_Store',
 'images',
 'intial_development',
 '.ipynb_checkpoints',
 'data',
 'overall',
 'targets_of_interest.csv']

In [8]:
# --- Observation Window UTC --- 

start = '2022-07-08 07:00'; end = '2022-07-08 10:00'

UTC_alpha = 1
UTC_beta = 2

station_timezone(end, UTC_alpha)
LST_window(start, end)


 LST Window for UTC 07:00 to 10:00 on the 2022-07-08 is:

 I-LOFAR: 1:33-4:33
LOFAR-SE: 2:52-5:53

 LST difference of ~79 minutes between stations


In [4]:
def dataframe_formatting(fraction_of_ot): 

    """
    Inputs: 
    fraction_of_ot - filters out the percentage of observing time for that falls under a given requirement e.g. 0.5 = 50%

    Outputs: 
    A csv. with possible targets during the given observation window dictated by the parameters of the 'setup' function. 

    """

    combined_targets_dataframe = []
    # --- Organising Dataframes --- 
    combined_targets_dataframe = pd.merge(Station_Alpha[0], Station_Beta[0], on = "target name") # -- merges on title column 
    combined_targets_dataframe = combined_targets_dataframe.rename(columns={"ever observable_x": "Ever Observable(%s)" % observatory_1.name, "ever observable_y": "Ever Observable(%s)" % observatory_2.name, "always observable_x" : "Always Observable(%s)" % observatory_1.name, "always observable_y" : "Always Observable(%s)" % observatory_2.name, "fraction of time observable_x":"Fraction of observable time (%s)" % observatory_1.name, "fraction of time observable_y":"Fraction of observable time (%s)" % observatory_2.name, "target name": "Name"}) # -- renaming cols. 
    combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Ever Observable(%s)' % observatory_1.name] == True]; combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Ever Observable(%s)' % observatory_2.name] == True] # -- removing targets that can't be observed during the window. 
    combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Fraction of observable time (%s)' % observatory_1.name] > fraction_of_ot];  combined_targets_dataframe = combined_targets_dataframe[combined_targets_dataframe['Fraction of observable time (%s)' % observatory_2.name] > fraction_of_ot] 
    combined_targets_dataframe = pd.merge(combined_targets_dataframe, Station_Beta[3], on = "Name") # - merges on target name. 
    combined_targets_dataframe.sort_values(by = 'RA', ascending = True, inplace = True) # - sorting values by ascending right ascension. 

    combined_targets_dataframe = combined_targets_dataframe[['Name', 'RA', 'DEC', 'Ever Observable(%s)' % observatory_1.name, 'Always Observable(%s)' % observatory_1.name, 'Ever Observable(%s)' % observatory_2.name, 'Always Observable(%s)' % observatory_2.name, 'Fraction of observable time (%s)' % observatory_1.name , 'Fraction of observable time (%s)' % observatory_2.name]]

    print('Number of possible targets:', len(combined_targets_dataframe))
    # --- Saving .csv ---
    combined_targets_dataframe.to_csv('data/target-lists/combined/Combined-Targets-%s.csv' % (Station_Alpha[2][2:8]), header = True, index  = False)

    return(combined_targets_dataframe)

In [15]:
# %%capture

Station_Alpha = []
Station_Beta = []

observatory_1 = ILOFAR
observatory_2 = ONSULA
catalog = 'targets_of_interest.csv'

# --- Computing Dataframes --- 
Station_Alpha = setup(station_timezone(start, UTC_alpha), station_timezone(end, UTC_alpha), 30, 90, 20, 0.016, observatory_1, catalog, True)
Station_Beta  = setup(station_timezone(start, UTC_beta), station_timezone(end, UTC_beta), 30, 90, 20,  0.016, observatory_2, catalog, False)

test = dataframe_formatting(0.8)
test

['2022-07-08 08:00:00.000' '2022-07-08 11:00:00.000']
Number of objects that do not meet the criteria of both stations: 273
Number of previously observed targets: 27
Number of objects avalible for observation: 546
['2022-07-08 09:00:00.000' '2022-07-08 12:00:00.000']
Number of objects that do not meet the criteria of both stations: 273
Number of previously observed targets: 27
Number of objects avalible for observation: 546
Number of possible targets: 546


,Name,RA,DEC,Ever Observable(I-LOFAR),Always Observable(I-LOFAR),Ever Observable(LOFAR-SE),Always Observable(LOFAR-SE),Fraction of observable time (I-LOFAR),Fraction of observable time (LOFAR-SE)
0,TIC346929661,0.065170,59.334681,True,True,True,True,1.0,1.0
137,TIC347013211,0.074617,54.934497,True,True,True,True,1.0,1.0
374,TIC396356111,0.173304,58.489439,True,True,True,True,1.0,1.0
373,TIC461387302,0.201453,66.650523,True,True,True,True,1.0,1.0
372,TIC403256331,0.283289,58.679253,True,True,True,True,1.0,1.0
...,...,...,...,...,...,...,...,...,...
177,TIC431585347,23.844936,50.354962,True,True,True,True,1.0,1.0
176,TIC378291076,23.880909,61.360260,True,True,True,True,1.0,1.0
175,TIC378355116,23.883226,60.481667,True,True,True,True,1.0,1.0
174,TIC470852531,23.982745,61.554089,True,True,True,True,1.0,1.0


In [17]:
step = 3
target_list = []

for i in range(0, 12):
    t = min(test['RA'], key=lambda x:abs(x - step))
    target_list.append(test.loc[test['RA'] == t]['Name']) 
    test = test.drop(test.loc[test['RA'] == t].index)
    step += 0.25
target_list

[390    TIC376353509
 Name: Name, dtype: object, 387    TIC458914949
 Name: Name, dtype: object, 383    TIC285674856
 Name: Name, dtype: object, 380    TIC73104318
 Name: Name, dtype: object, 379    TIC102672709
 Name: Name, dtype: object, 377    TIC103195323
 Name: Name, dtype: object, 306    TIC8400842
 Name: Name, dtype: object, 341    TIC85031598
 Name: Name, dtype: object, 342    TIC470381900
 Name: Name, dtype: object, 299    TIC241225337
 Name: Name, dtype: object, 298    TIC138168780
 Name: Name, dtype: object, 297    TIC396740648
 Name: Name, dtype: object]